# Setup

In [ ]:
!pip3 install git+https://github.com/huggingface/diffusers --upgrade
!pip3 install accelerate
!pip3 install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Train Model

In [ ]:
MODEL_NAME="wavymulder/portraitplus"
INSTANCE_DIR="/content/drive/MyDrive/AIFootage/muqing/dreambooth"
OUTPUT_DIR="/content/drive/MyDrive/AIFootage/muqing/dreambooth/output"
INSTANCE_KEY = "mbai"
CLASS_KEY = "Bai"


In [ ]:
if os.path.exists(OUTPUT_DIR) == False:
    os.makedirs(OUTPUT_DIR)

In [ ]:
!python3 "train_dreambooth_lora.py" --pretrained_model_name_or_path={MODEL_NAME}  --instance_data_dir={INSTANCE_DIR} --output_dir={OUTPUT_DIR} --instance_prompt="A photo of {INSTANCE_KEY}" --class_data_dir="A photo of {CLASS_KEY}" --resolution=512 --train_batch_size=1 --gradient_accumulation_steps=1 --checkpointing_steps=50000 --learning_rate=1e-4 --lr_scheduler="constant" --lr_warmup_steps=0 --max_train_steps=500 --validation_prompt="portrait+ style A photo of {INSTANCE_KEY} on the beach" --validation_epochs=50000 --seed="42"

# Generate Images with Trained Model

In [ ]:
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
import torch
import os

pipe = DiffusionPipeline.from_pretrained("wavymulder/portraitplus", torch_dtype=torch.float16)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.to("cuda")
generator = torch.Generator(device="cuda").manual_seed(42)
pipe.load_lora_weights("portraitp2/pytorch_lora_weights.bin")
imagesToGenerate = 1
movies = ["A pizza place"]
personName = "mbai"
for movie in movies:
    images = pipe("A photo of "+personName+" in " + movie + " eating pizza. Fat. Chubby=3:5. OilySkin=4:5. Acne=4:5. Eyebags=5:5",negative_prompt=" blurry, women", num_inference_steps=55, guidance_scale=7, num_images_per_prompt=imagesToGenerate, generator=generator).images
    for indx, image_ in enumerate(images):
        outpath = "output/" +personName.replace(" ", "_")+"_"+ movie.replace(" ", "_")
        if not os.path.exists(outpath):
            os.makedirs(outpath)
        image_.save(outpath + "/"+ personName.replace(" ", "_")+"_" + movie.replace(" ", "_") + "_"+str(indx)+".png")

# Save the image